In [1]:
# ( timestamp > 1564272000000 and timestamp < 1567468800000 )  ?  NaN  :  pm25_raw

# ( "mqtt:nesta-7".equals("$node") and (timestamp > 1564272000000 and
# timestamp < 1567468800000 )) ? NaN : pm25_raw

In [2]:
from datetime import datetime
from datetime import timezone
from dateutil.parser import parse

In [3]:
from get_aq_data import ID_to_name

In [4]:
def to_timestamp(dt):
    return int(dt.replace(tzinfo=timezone.utc).timestamp()*1000)

def from_timestamp(timestamp):
    return datetime.utcfromtimestamp(int(timestamp) / 1000)

In [20]:
def get_timestamp_statement(value):
    if len(value) == 2:
        start_timestamp = to_timestamp(parse(value[0]))
        end_timestamp = to_timestamp(parse(value[1]))
        timestamp_statement = f'(timestamp > {start_timestamp} and timestamp < {end_timestamp})'
    elif len(value) == 1:
        end_timestamp = to_timestamp(parse(value[0]))
        timestamp_statement  = f'(timestamp < {end_timestamp})'
    
    return timestamp_statement

In [37]:
def print_individual_filters(data):
    for name, time_specs in data.items():
        sensor_name = ID_to_name[name.split(':')[1]]

        if len(time_specs) == 1:
            timestamp_statement = get_timestamp_statement(time_specs[0])
            output = timestamp_statement
        else:
            timestamp_statements = [get_timestamp_statement(time_spec) for time_spec in time_specs]
            output = f'( {" or ".join(timestamp_statements)} )'

        print(f'{sensor_name}: {output} ? NaN : pm25_raw')

In [38]:
def print_overall_filter(data):
    node_statements = []
    for name, time_specs in data.items():

        if len(time_specs) == 1:
            timestamp_statement = get_timestamp_statement(time_specs[0])
            output = timestamp_statement
        else:
            timestamp_statements = [get_timestamp_statement(time_spec) for time_spec in time_specs]
            output = f'( {" or ".join(timestamp_statements)} )'

        node_statements.append(f'("{name}".equals("$node") and {output})')

    print(" or ".join(node_statements) + " ? NaN : pm25_raw")

In [ ]:
# ( timestamp > 1571097600000 and timestamp < 1571918400000 )  ?  NaN  :  pm25_raw

In [43]:
from_timestamp(1571918400000)

datetime.datetime(2019, 10, 24, 12, 0)

In [44]:
data = {
        'mqtt:nesta-7': [['2019-07-28', '2019-09-03']],
        'mqtt:nesta-9': [['2019-10-15']],
        'mqtt:nesta-4': [['2019-10-15', '2019-11-06']]
        # Example below for multiple periods
        #'mqtt:nesta-2-1': [('2019-10-01 10:00', '2019-10-01 14:00'), ('2019-09-30 22:00', '2019-10-01 03:00')]
       }

In [45]:
print_individual_filters(data)

St Denys Rd: (timestamp > 1564272000000 and timestamp < 1567468800000) ? NaN : pm25_raw
Riverside: (timestamp < 1571097600000) ? NaN : pm25_raw
Horseshoe Bridge: (timestamp > 1571097600000 and timestamp < 1572998400000) ? NaN : pm25_raw


In [46]:
print_overall_filter(data)

("mqtt:nesta-7".equals("$node") and (timestamp > 1564272000000 and timestamp < 1567468800000)) or ("mqtt:nesta-9".equals("$node") and (timestamp < 1571097600000)) or ("mqtt:nesta-4".equals("$node") and (timestamp > 1571097600000 and timestamp < 1572998400000)) ? NaN : pm25_raw
